In [ ]:
####
#### 해당 파일은 데이터셋 전처리 및 파일 추출 전용 입니다.
####

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

import wordcloud
from wordcloud import WordCloud
import json
import folium
%matplotlib inline
# 한글 설정
# pip install koreanize_matplotlib
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
root = 'C:/workspace/python/project/data/'

# 구글드라이브 : https://drive.google.com/drive/folders/1zIzm1o8-3uxcWSU2DoWpB8aV0Oxdfz_P?usp=sharing

In [18]:
##################################### 데이터 로드 ######################################
'''
서울시 집값 데이터 프레임
'''
seoul_2021_df = pd.read_csv(root + '부동산_Data/부동산_2021.csv', encoding = 'cp949')
seoul_2022_df = pd.read_csv(root + '부동산_Data/부동산_2022.csv', encoding = 'cp949')
seoul_2023_df = pd.read_csv(root + '부동산_Data/부동산_2023.csv', encoding = 'cp949')

# dict_keys(['2019년 12월 31일 기준', '2020년 12월 31일 기준', '2021년 12월 31일 기준', '2022년 12월 31일 기준', '2023년 12월 4일 기준'])
all_bus_stop_dict = pd.read_excel(root + '교통_Data/버스_정류소현황(2019~2023년).xlsx', sheet_name=None)
bus_stop_address_df = pd.read_excel(root + '교통_Data/서울시버스정류소위치정보(20241110).xlsx')

'''
유통업체 데이터 프레임
'''
distributor_2021_df = pd.read_excel(root + '유통업체_Data/유통업체현황_2021.xlsx', header=None)
distributor_2022_df = pd.read_excel(root + '유통업체_Data/유통업체현황_2022.xlsx', header=None)
distributor_2023_df = pd.read_excel(root + '유통업체_Data/유통업체현황_2023.xlsx', header=None)

'''
버스 정류장 데이터 프레임
'''
bus_stop_2021_df = all_bus_stop_dict['2021년 12월 31일 기준']
bus_stop_2022_df = all_bus_stop_dict['2022년 12월 31일 기준']
bus_stop_2023_df = all_bus_stop_dict['2023년 12월 4일 기준']

'''
지하철 데이터 프레임
'''
train_2021_df = pd.read_csv(root + '교통_Data/서울시 지역별 지하철역 정보_2021.csv', encoding = 'cp949')
train_2022_df = pd.read_csv(root + '교통_Data/서울시 지역별 지하철역 정보_2022.csv', encoding = 'cp949')
train_2023_df = pd.read_csv(root + '교통_Data/서울시 지역별 지하철역 정보_2023.csv', encoding = 'cp949')

'''
인구수 데이터 프레임
'''
population_df = pd.read_csv(root + '인구수_Data/인구수.csv', encoding = 'cp949')

'''
공원 데이터 프레임
'''
park_2021_df = pd.read_excel(root + '공원_Data/공원현황_2021.xlsx', header=None)
park_2022_df = pd.read_excel(root + '공원_Data/공원현황_2022.xlsx', header=None)
park_2023_df = pd.read_excel(root + '공원_Data/공원현황_2023.xlsx', header=None)

'''
의료기관 데이터 프레임
'''
medical_2021_df = pd.read_excel(root + '의료기관_Data/의료기관_2021.xlsx', header=None)
medical_2022_df = pd.read_excel(root + '의료기관_Data/의료기관_2022.xlsx', header=None)
medical_2023_df = pd.read_excel(root + '의료기관_Data/의료기관_2023.xlsx', header=None)

'''
개발계획 데이터 프레임
'''
develop_2021_df = pd.read_csv(root + '부동산_Data/개발계획2021.csv', encoding='cp949', header=None)
develop_2022_df = pd.read_csv(root + '부동산_Data/개발계획2022.csv', encoding='cp949', header=None)
develop_2023_df = pd.read_csv(root + '부동산_Data/개발계획2023.csv', encoding='cp949', header=None)

'''
건축거래 현황 데이터 프레임
'''
house_transaction_df = pd.read_csv(root + '주거실태_Data/행정구역별_건축물거래현황_20241126100629.csv', encoding='cp949', header=None)

'''
수요/공급지수 데이터 프레임
'''
buysell_df = pd.read_csv(root + '수요공급_Data/지역별_매매동향.csv', header=None, encoding='cp949')

In [19]:
##################################### 전처리 함수 작성 #######################################
'''
서울시 집값 전처리 함수 (행정구 별 평균시세)
'''
def precleaning_seoul(df) :
  df = df.groupby(['접수연도', '자치구명'])['물건금액(만원)'].mean().reset_index()
  df = df.rename(columns = {'접수연도' : '연도', '자치구명' : '구', '물건금액(만원)' : '평균시세'})
  df['평균시세'] = round(df['평균시세'], 2)
  return df

'''
유통업체 전처리 함수
'''
def precleaning_distributor(df) :
  # 컬럼명 지정
  df.columns = df.iloc[1] + '_' + df.iloc[3]
  df.columns = df.columns.str.strip()
  # 쓸모없는 행 삭제
  df = df.iloc[4:].reset_index(drop=True)
  # 쓸모없는 열 삭제
  df = df.drop(columns = ['동별(1)_동별(1)'])
  
  # 컬럼명 보기 쉽게 변경
  df = df.rename(columns = {'동별(2)_동별(2)' : '구'})
  columns = []
  for column in df.columns : 
    column = column.split(' (')
    columns.append(column[0])
  df.columns = columns
  return df

'''
버스 전처리 함수
'''
def precleaning_bus(df) :
  # 연도를 보기쉽게 바꾸기 위해 앞 4글자만 추출
  year = df['기준일'].astype(str).str[:4]
  
  # 추출한 값으로 변경
  df['기준일'] = year
  
  # 컬럼명 통일
  df = df.rename(columns = {'기준일' : '연도', '행정구명' : '구'})
  
  # 버스 정보와 위치정보 병합
  df = pd.merge(df, bus_stop_address_df[['ARS_ID', 'X좌표', 'Y좌표']], 
                              left_on='ARS-ID', right_on='ARS_ID', how='left')
  # ARS_ID 중복 컬럼 제거
  df.drop(columns=['ARS_ID'], inplace=True)

  # 이상한 값 변경
  idx = df[df['X좌표'] == 126.457230].index[0]

  df.loc[idx, 'X좌표'] = 127.037725
  df.loc[idx, 'Y좌표'] = 37.613705

  df['정류소명'] = df['정류소명'].replace({'\u2024': '.'}, regex=True)
  return df

'''
지하철 전처리 함수
'''
def precleaning_train(df) :
  df.columns = ['구', '해당역(호선)', '역개수']
  filtered_df = df[df['구'].isin(seoul_2021_avg_df['구'])]
  return filtered_df

'''
인구수 전처리 함수
'''
def precleaning_population(df, choice) :
  year = df.columns.str.contains(choice)
  year[0] = True
  
  tmp_df = df.loc[:, year]
  columns_name = tmp_df.iloc[0].values
  tmp_df.columns = columns_name
  tmp_df = tmp_df.drop(0)
  tmp_df = tmp_df.rename(columns = {'행정구역별(읍면동)' : '구'})
  columns_name = tmp_df.columns[1:]
  
  for idx, column in enumerate(columns_name) :
    tmp_df = tmp_df.rename(columns = {column : column.split(' ')[0]})
  
  return tmp_df

'''
공원 데이터 전처리
'''
def preclearning_park(df):
  # 컬럼명 변경
  # 2 ~ 5 번 컬럼명을 첫 번째 행 데이터로 변경
  columns_name = df.columns[1:5]
  for column in columns_name :
    name = df[column][1] + '_' + df[column][4]
    df = df.rename(columns = {column : name})
  
  # 5 ~ 10 번 컬럼명을 1, 4 번째 행 데이터로 변경
  columns_name = df.columns[5 : 11]
  for column in columns_name :
    name = df[column][1] + '_' + df[column][4]
    df = df.rename(columns = {column : name})  
  
  
  # 11번 이후 컬럼명을 3, 4번째 행 데이터로 변경
  columns_name = df.columns[11:]
  for column in columns_name :
    name = df[column][3] + '_' + df[column][4]
    df = df.rename(columns = {column : name})
  
  # 컬럼명 변경
  df = df.rename(columns={'자치구별(2)_자치구별(2)' : '구'})
  
  # 필요없는 컬럼 삭제
  df = df.drop(columns=0)
  
  #필요 없는 행 삭제
  df = df.drop(index=[0, 1, 2, 3, 4])
  
  # 인덱스 리셋
  df = df.reset_index()
  df = df.drop(columns=['index'])

  # 년도 컬럼 추가
  df = pd.DataFrame(df)
  df['년도'] = 2021
  df = pd.DataFrame(df)
  df['년도'] = 2022
  df = pd.DataFrame(df)
  df['년도'] = 2023

  cols = ['년도'] + [col for col in df.columns if col != '년도']
  df = df[cols]
  return df

'''
의료기관 데이터 전처리
'''
def precleaning_medical(df, year) :
  # 컬럼명 지정
  df.columns = df.iloc[2] + '_' +  df.iloc[3]
 
  # 쓸모없는 행 삭제
  df = df[4:]
  
  # 슬모없는 열 삭제
  df = df.drop(columns = '자치구별(1)_자치구별(1)')

  if year > 2022:
    df = df.drop(columns = '한센병원_병원수')
    
  # 보기쉽게 열이름 바꿈
  df = df.rename(columns = {'자치구별(2)_자치구별(2)' : '구'})
  condition1 = df.columns.str.contains('_병원수')
  selected_columns = df.loc[:, condition1]  # 열 기준으로 선택
  columns = []
  columns.append('구')

  # 데이터에 있는 병원 계열로 컬럼 지정
  for column in selected_columns.columns :
    columns.append(column)
  df = df[columns]

  # 행 지운 후 인덱스 초기화
  df = df.reset_index()
  df = df.drop(columns=['index'])
  
  return df

'''
개발계획 데이터 전처리
'''
def precleaning_develop(df):
  # 컬럼명 지정
  df.columns = df.iloc[1] + '_' + df.iloc[2]
  
  # 불필요한 데이터 삭제
  df = df.drop(index=[0, 1, 2])

  
  # 서울특별시 데이터만 추출
  df = df[df['소재지(시군구)별(1)_소재지(시군구)별(1)'] == '서울특별시']

  # 컬러명 변경 
  df = df.rename(columns ={'소재지(시군구)별(2)_소재지(시군구)별(2)' : '구'})
  df = df.rename(columns ={'소재지(시군구)별(1)_소재지(시군구)별(1)' : '-'})
  # 인덱스 초기화
  df = df.reset_index()
  df = df.drop(columns=['index'])

  
  return df

'''
건축거래 현황 데이터 전처리리
'''
def precleaning_transaction(df, year) :
  copy_df = df.copy()
  copy_df.columns = copy_df.iloc[0] + '_' + copy_df.iloc[1]
  copy_df = copy_df.iloc[2:, 1:]
  copy_df = copy_df.rename(columns = {'행정구역별(2)_행정구역별(2)' : '구'})

  condition1 = copy_df.columns.str.contains(year)
  select_columns = copy_df.loc[:, condition1].columns
  columns = []
  columns.append('구')
  for column in select_columns :
    columns.append(column)
  
  selected_df = copy_df[columns]
  # Melt 함수를 사용하여 열을 행으로 변환
  df_melted = selected_df.melt(id_vars=['구'], var_name='월', value_name='값')
  
  # 월별 동(호)수와 면적을 구분하기 위해 컬럼을 분리
  df_melted[['연도_월', '항목']] = df_melted['월'].str.split('_', expand=True)
  
  # 항목이 동(호)수인지 면적인지에 따라 값 분리
  df_melted = df_melted.pivot_table(index=['구', '연도_월'], columns='항목', values='값', aggfunc='first').reset_index()
  return df_melted

'''
수급동급 전처리 함수
'''
def precleaning_buysell(df):
  
  df[['종로구', '중구', '용산구']] = pd.concat([df[1]] *3, axis=1).values
  df[['성동구', '광진구', '동대문구', '중랑구', '성북구','강북구','도봉구','노원구']] = pd.concat([df[2]] *8, axis=1).values
  df[['은평구', '서대문구', '마포구']] = pd.concat([df[3]] *3, axis=1).values
  df[['양천구', '강서구', '구로구','금천구','영등포구','동작구','관악구']] = pd.concat([df[4]] *7, axis=1).values
  df[['서초구', '강남구', '송파구','강동구']] = pd.concat([df[5]] *4, axis=1).values
  df = df.drop(columns=[1,2,3,4,5])
  df = df.drop(index=[0,1])
  df = df.rename(columns={0 : '년월'})
  df = df.reset_index()
  df = df.drop(columns=['index'])
  return df
'''
수급동급 '구'마다 '월'단위로 수급동급 추출
'''
def cleaning_data(df):
  df['년월'] = df['년월'].astype(str)
  df['년월'] = df['년월'].str.split('-').str[1]
  df = df.melt(id_vars=['년월'], var_name='구', value_name='수급등급')
  df.rename(columns={'년월':'월'}, inplace=True)
  return df

In [20]:
################################## 전처리 함수 적용 ###################################

    # 서울시 집값 전처리 함수 적용
seoul_2021_avg_df = precleaning_seoul(seoul_2021_df)
seoul_2022_avg_df = precleaning_seoul(seoul_2022_df)
seoul_2023_avg_df = precleaning_seoul(seoul_2023_df)

    # 유통업체 전처리 함수 적용
distributor_2021_df = precleaning_distributor(distributor_2021_df)
distributor_2022_df = precleaning_distributor(distributor_2022_df)
distributor_2023_df = precleaning_distributor(distributor_2023_df)

    # 버스 전처리 함수 적용
bus_stop_2021_df = precleaning_bus(bus_stop_2021_df)
bus_stop_2022_df = precleaning_bus(bus_stop_2022_df)
bus_stop_2023_df = precleaning_bus(bus_stop_2023_df)

    # 지하철 전처리 함수 적용
train_2021_df.drop(columns = '연번', inplace= True)
train_2022_df.drop(columns = '연번', inplace= True)
train_2021_df = precleaning_train(train_2021_df)
train_2022_df = precleaning_train(train_2022_df)
train_2023_df = precleaning_train(train_2023_df)

    # 인구수 전처리 함수 적용
population_2021_df = precleaning_population(population_df, '2021')
population_2022_df = precleaning_population(population_df, '2022')
population_2023_df = precleaning_population(population_df, '2023')

    # 공원 전처리 함수 적용
park_2021_df = preclearning_park(park_2021_df)
park_2022_df = preclearning_park(park_2022_df)
park_2023_df = preclearning_park(park_2023_df)
# 년도 컬럼 추가
park_2021_df = pd.DataFrame(park_2021_df)
park_2021_df['년도'] = 2021
park_2022_df = pd.DataFrame(park_2022_df)
park_2022_df['년도'] = 2022
park_2023_df = pd.DataFrame(park_2023_df)
park_2023_df['년도'] = 2023
cols = ['년도'] + [col for col in park_2021_df.columns if col != '년도']
cols = ['년도'] + [col for col in park_2022_df.columns if col != '년도']
cols = ['년도'] + [col for col in park_2023_df.columns if col != '년도']
park_2021_df = park_2021_df[cols]
park_2022_df = park_2022_df[cols]
park_2023_df = park_2023_df[cols]
# 년도, 구 , 합계 컬러만 추출
park_2021_col = park_2021_df.loc[1:, ['년도', '구', '합계_공원수 (개소)']]
park_2022_col = park_2022_df.loc[1:, ['년도', '구', '합계_공원수 (개소)']]
park_2023_col = park_2023_df.loc[1:, ['년도', '구', '합계_공원수 (개소)']]

    # 의료기관 전처리 함수 적용
medical_2021_df = precleaning_medical(medical_2021_df, 2021)
medical_2022_df = precleaning_medical(medical_2022_df, 2022)
medical_2023_df = precleaning_medical(medical_2023_df, 2023)
# 년도 컬럼 추가
medical_2021_df = pd.DataFrame(medical_2021_df)
medical_2021_df['년도'] = 2021
medical_2022_df = pd.DataFrame(medical_2022_df)
medical_2022_df['년도'] = 2022
medical_2023_df = pd.DataFrame(medical_2023_df)
medical_2023_df['년도'] = 2023
cols = ['년도'] + [col for col in medical_2021_df.columns if col != '년도']
cols = ['년도'] + [col for col in medical_2022_df.columns if col != '년도']
cols = ['년도'] + [col for col in medical_2023_df.columns if col != '년도']
medical_2021_df = medical_2021_df[cols]
medical_2022_df = medical_2022_df[cols]
medical_2023_df = medical_2023_df[cols]
# 의료기관 컬럼 추출
medical_2021_col = medical_2021_df[['년도','구','소계_병원수']]
medical_2022_col = medical_2022_df[['년도','구','소계_병원수']]
medical_2023_col = medical_2023_df[['년도','구','소계_병원수']]

    # 개발계획 데이터 전처리 함수 적용
develop_2021_df=precleaning_develop(develop_2021_df)
develop_2022_df=precleaning_develop(develop_2022_df)
develop_2023_df=precleaning_develop(develop_2023_df)
# 2021 개발계획 년도 추가 
develop_2021_tmp = pd.DataFrame(develop_2021_df)
develop_2021_tmp['년도'] = 2021
# 2022 개발계획 년도 추가 
develop_2022_tmp = pd.DataFrame(develop_2022_df)
develop_2022_tmp['년도'] = 2022
# 2023 개발계획 년도 추가 
develop_2023_tmp = pd.DataFrame(develop_2023_df)
develop_2023_tmp['년도'] = 2023
# 년도 컬럼 맨 앞으로
cols = ['년도'] + [col for col in develop_2021_tmp.columns if col != '년도']
develop_2021_tmp = develop_2021_tmp[cols]
cols = ['년도'] + [col for col in develop_2022_tmp.columns if col != '년도']
develop_2022_tmp = develop_2022_tmp[cols]
cols = ['년도'] + [col for col in develop_2023_tmp.columns if col != '년도']
develop_2023_tmp = develop_2023_tmp[cols]
# 년도, 구 , 합계 컬러만 추출
develop_2021_col = develop_2021_tmp.loc[1:, ['년도', '구', '계_구역수 (개)']]
develop_2022_col = develop_2022_tmp.loc[1:, ['년도', '구', '계_구역수 (개)']]
develop_2023_col = develop_2023_tmp.loc[1:, ['년도', '구', '계_구역수 (개)']]

    # 건축거래 현황 전처리 함수 적용
house_transaction_2021_df = precleaning_transaction(house_transaction_df, '2021')
house_transaction_2021_df['년도'] = '2021'
house_transaction_2022_df = precleaning_transaction(house_transaction_df, '2022')
house_transaction_2022_df['년도'] = '2022'
house_transaction_2023_df = precleaning_transaction(house_transaction_df, '2023')
house_transaction_2023_df['년도'] = '2023'
# 데이터프레임 병합
house_transaction_combined = pd.concat([house_transaction_2021_df, house_transaction_2022_df, house_transaction_2023_df], ignore_index=True)
house_transaction_combined = house_transaction_combined.rename(columns={'동(호)수 (동(호)수)':'동(호)수'})
house_transaction_combined['동(호)수']    = house_transaction_combined['동(호)수'].astype(int)
house_transaction_combined['면적 (천㎡)'] = house_transaction_combined['면적 (천㎡)'].astype(int)
house_transaction_grouped = house_transaction_combined.groupby(['년도', '구']).agg({'동(호)수': 'sum',
                                                                                    '면적 (천㎡)': 'sum'}).reset_index()
# 데이터 준비
grouped_data = house_transaction_grouped.copy()
grouped_data = grouped_data[grouped_data['구'] != '소계']
grouped_data_2021 = grouped_data[grouped_data['년도'] == '2021']
grouped_data_2022 = grouped_data[grouped_data['년도'] == '2022']
grouped_data_2023 = grouped_data[grouped_data['년도'] == '2023']

    # 수급동급 전처리 함수 적용
buysell_df = precleaning_buysell(buysell_df)
# '''
# 데이터 값이 79 이하 0, 80~120 은 1, 121~ 2 로 변경
# '''
# # 데이터 타입 변경
# buysell_df.iloc[:,1:] = buysell_df.iloc[:,1:].astype(float)

# """ 람다식 
# buysell_df.iloc[:, 1:] = buysell_df.iloc[:, 1:].applymap(
#    lambda x: 0 if x <= 79 else (1 if x <= 120 else 2)) """

# # 조건과 대응되는 값 설정
# conditions = [
#     (buysell_df.iloc[:, 1:] <= 79),
#     (buysell_df.iloc[:, 1:] >= 80) & (buysell_df.iloc[:, 1:] <= 120),
#     (buysell_df.iloc[:, 1:] >= 121)
# ]
# choices = [0, 1, 2]

# # 조건에 따라 값 변경
# buysell_df.iloc[:, 1:] = np.select(conditions, choices)

# buysell_df
# 년월 데이터를 date화 함
buysell_df['년월'] = pd.to_datetime(buysell_df['년월'], errors='coerce')

# 년도별로 분리
buysell_2021_df = buysell_df[buysell_df['년월'].dt.year == 2021]
buysell_2022_df = buysell_df[buysell_df['년월'].dt.year == 2022]
buysell_2023_df = buysell_df[buysell_df['년월'].dt.year == 2023]
buysell_2024_df = buysell_df[buysell_df['년월'].dt.year == 2024]
# 추출 함수 실행
buysell_2021_df = cleaning_data(buysell_2021_df)
buysell_2022_df = cleaning_data(buysell_2022_df)
buysell_2023_df = cleaning_data(buysell_2023_df)
buysell_2024_df = cleaning_data(buysell_2024_df)

In [24]:
################################ 전처리 적용 데이터프레임 csv 저장 ##########################

# '''
# 서울시 집값 / 부동산
# '''
# seoul_2021_avg_df.to_csv('전처리_부동산_2021.csv', index = False, encoding = 'cp949')
# seoul_2022_avg_df.to_csv('전처리_부동산_2022.csv', index = False, encoding = 'cp949')
# seoul_2023_avg_df.to_csv('전처리_부동산_2023.csv', index = False, encoding = 'cp949')

# '''
# 버스 정류장 수
# '''
# bus_stop_2021_df.to_csv('전처리_버스수_2021.csv', index = False, encoding = 'cp949')
# bus_stop_2022_df.to_csv('전처리_버스수_2022.csv', index = False, encoding = 'cp949')
# bus_stop_2023_df.to_csv('전처리_버스수_2023.csv', index = False, encoding = 'cp949')

# '''
# 지하철 역 개수
# '''
# train_2021_df.to_csv('전처리_지하철수_2021.csv', index = False, encoding = 'cp949')
# train_2022_df.to_csv('전처리_지하철수_2022.csv', index = False, encoding = 'cp949')
# train_2023_df.to_csv('전처리_지하철수_2023.csv', index = False, encoding = 'cp949')

# '''
# 공원 개수
# '''
# park_2021_col.to_csv('전처리_공원수_2021.csv', index = False, encoding = 'cp949')
# park_2022_col.to_csv('전처리_공원수_2022.csv', index = False, encoding = 'cp949')
# park_2023_col.to_csv('전처리_공원수_2023.csv', index = False, encoding = 'cp949')

# '''
# 유통업체 수
# '''
# distributor_2021_df.to_csv('전처리_유통업체_2021.csv', index = False, encoding = 'cp949')
# distributor_2022_df.to_csv('전처리_유통업체_2022.csv', index = False, encoding = 'cp949')
# distributor_2023_df.to_csv('전처리_유통업체_2023.csv', index = False, encoding = 'cp949')

# '''
# 의료기관 / 병원 수
# '''
# medical_2021_col.to_csv('전처리_병원수_2021.csv', index = False, encoding = 'cp949')
# medical_2022_col.to_csv('전처리_병원수_2022.csv', index = False, encoding = 'cp949')
# medical_2023_col.to_csv('전처리_병원수_2023.csv', index = False, encoding = 'cp949')

# '''
# 개발 계획
# '''
# develop_2021_col.to_csv('전처리_개발계획_2021.csv', index = False, encoding = 'cp949')
# develop_2022_col.to_csv('전처리_개발계획_2022.csv', index = False, encoding = 'cp949')
# develop_2023_col.to_csv('전처리_개발계획_2023.csv', index = False, encoding = 'cp949')

# '''
# 인구 수
# '''
# population_2021_df.to_csv('전처리_인구수_2021.csv', index = False, encoding = 'cp949')
# population_2022_df.to_csv('전처리_인구수_2022.csv', index = False, encoding = 'cp949')
# population_2023_df.to_csv('전처리_인구수_2023.csv', index = False, encoding = 'cp949')

# '''
# 주거실태 / 거래량 / 건축거래 현황
# '''
# grouped_data_2021.to_csv('전처리_거래량_2021.csv', index = False, encoding = 'cp949')
# grouped_data_2022.to_csv('전처리_거래량_2022.csv', index = False, encoding = 'cp949')
# grouped_data_2023.to_csv('전처리_거래량_2023.csv', index = False, encoding = 'cp949')

# '''
# 수요/공급 지수 
# '''
# buysell_2021_df.to_csv('전처리_수요공급지수_2021.csv', index=False, encoding='cp949')
# buysell_2022_df.to_csv('전처리_수요공급지수_2022.csv', index=False, encoding='cp949')
# buysell_2023_df.to_csv('전처리_수요공급지수_2023.csv', index=False, encoding='cp949')
# buysell_2024_df.to_csv('전처리_수요공급지수_2024.csv', index=False, encoding='cp949')